# AI City View - Vision API Server (Colab)

This notebook runs the AI_City_View computer vision pipeline as a REST API server,
accessible via ngrok tunnel. Use the public URL in GreenSVC Settings as `VISION_API_URL`.

**Requirements:** GPU runtime (T4 or better). Go to Runtime > Change runtime type > GPU.

**Steps:**
1. Run all cells in order
2. Copy the ngrok public URL printed in Cell 5
3. Paste it into GreenSVC Settings as the Vision API URL

In [ ]:
#@title Cell 1: Check GPU & Install PyTorch CUDA
import torch
import subprocess

if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_mem / (1024**3)
    print(f"GPU detected: {gpu_name} ({gpu_mem:.1f} GB)")
    print(f"CUDA version: {torch.version.cuda}")
    print(f"PyTorch version: {torch.__version__}")
else:
    print("WARNING: No GPU detected!")
    print("Go to Runtime > Change runtime type > GPU (T4 recommended)")
    raise RuntimeError("GPU required for vision pipeline")

In [ ]:
#@title Cell 2: Clone repo & install dependencies
import os

# Option A: Clone from GitHub (update URL to your repo)
REPO_URL = ""  #@param {type:"string"}
REPO_DIR = "/content/AI_City_View"

if REPO_URL:
    if os.path.exists(REPO_DIR):
        print(f"Repo already exists at {REPO_DIR}, pulling latest...")
        !cd {REPO_DIR} && git pull
    else:
        !git clone {REPO_URL} {REPO_DIR}
    print(f"Repo ready at {REPO_DIR}")
else:
    # Option B: Upload manually
    # Upload your AI_City_View folder to /content/AI_City_View
    if not os.path.exists(REPO_DIR):
        print("No REPO_URL set and no local repo found.")
        print("Either set REPO_URL above, or upload files to /content/AI_City_View")
        print("You can use: Files panel (left) > Upload, or mount Google Drive.")
    else:
        print(f"Using existing repo at {REPO_DIR}")

# Install requirements
if os.path.exists(f"{REPO_DIR}/requirements.txt"):
    !pip install -q -r {REPO_DIR}/requirements.txt
    print("Core dependencies installed")

# Install xformers for memory-efficient attention (optional but recommended)
!pip install -q xformers 2>/dev/null || echo "xformers install skipped (optional)"

print("Done!")

In [ ]:
#@title Cell 3: Install server dependencies
!pip install -q fastapi uvicorn python-multipart pyngrok
print("Server dependencies installed: fastapi, uvicorn, python-multipart, pyngrok")

In [ ]:
#@title Cell 4: Configure ngrok
# Get your free auth token at https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_AUTH_TOKEN = ""  #@param {type:"string"}

if not NGROK_AUTH_TOKEN:
    print("WARNING: No ngrok auth token set.")
    print("Get a free token at: https://dashboard.ngrok.com/get-started/your-authtoken")
    print("Then paste it above and re-run this cell.")
else:
    from pyngrok import ngrok
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    print("ngrok auth token configured")

In [ ]:
#@title Cell 5: Start server with ngrok tunnel
import os
import sys
import threading
import time

REPO_DIR = "/content/AI_City_View"
PORT = 8000

# Add repo to path
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)
os.chdir(REPO_DIR)

# Start ngrok tunnel
from pyngrok import ngrok
public_url = ngrok.connect(PORT, "http").public_url

print("=" * 60)
print(f"  Vision API public URL: {public_url}")
print(f"  Health check:          {public_url}/health")
print(f"  Analyze endpoint:      {public_url}/analyze")
print("=" * 60)
print()
print("Copy the URL above and paste it into GreenSVC Settings")
print("as the VISION_API_URL.")
print()
print("Starting uvicorn server (model loading may take 30-60s)...")
print()

# Start uvicorn in a background thread so the cell doesn't block
def run_server():
    import uvicorn
    uvicorn.run(
        "server:app",
        host="0.0.0.0",
        port=PORT,
        workers=1,
        log_level="info",
    )

server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

# Wait for server to be ready
import urllib.request
for i in range(120):  # Wait up to 2 minutes for model loading
    try:
        resp = urllib.request.urlopen(f"http://localhost:{PORT}/health", timeout=2)
        if resp.status == 200:
            print(f"Server is ready! (took ~{i}s)")
            break
    except Exception:
        time.sleep(1)
else:
    print("Server may still be loading models. Check logs above.")

print(f"\nPublic URL: {public_url}")

In [ ]:
#@title Cell 6: Test the API (optional)
import requests
import json
import os

# Use a test image from the repo or upload your own
REPO_DIR = "/content/AI_City_View"
BASE_URL = f"http://localhost:8000"  # Use localhost for testing (faster than ngrok)

# 1. Health check
print("1. Health check:")
resp = requests.get(f"{BASE_URL}/health")
print(json.dumps(resp.json(), indent=2))
print()

# 2. Get config
print("2. Semantic config (first 3 classes):")
resp = requests.get(f"{BASE_URL}/config")
config = resp.json()
for item in config[:3]:
    print(f"  - {item['name']} (color: {item['color']})")
print(f"  ... ({len(config)} classes total)")
print()

# 3. Analyze a test image
# Find a test image in the repo
test_images = [f for f in os.listdir(REPO_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
if test_images:
    test_image = os.path.join(REPO_DIR, test_images[0])
    print(f"3. Analyzing test image: {test_images[0]}")

    # Build minimal request
    request_data = {
        "image_id": "test_001",
        "semantic_classes": [item["name"] for item in config],
        "semantic_countability": [item["countable"] for item in config],
        "openness_list": [item["openness"] for item in config],
    }

    with open(test_image, "rb") as f:
        resp = requests.post(
            f"{BASE_URL}/analyze",
            files={"file": (os.path.basename(test_image), f, "image/jpeg")},
            data={"request_data": json.dumps(request_data)},
            timeout=600,
        )

    result = resp.json()
    print(f"  Status: {result['status']}")
    print(f"  Job ID: {result.get('job_id', 'N/A')}")
    print(f"  Detected classes: {result.get('detected_classes', 'N/A')}")
    print(f"  Images returned: {len(result.get('images', {}))}")
    print(f"  Processing time: {result.get('processing_time', 'N/A')}s")
    print(f"  Download URL: {result.get('download_url', 'N/A')}")

    if result.get('class_statistics'):
        print("  Top classes:")
        sorted_classes = sorted(
            result['class_statistics'].items(),
            key=lambda x: x[1]['percentage'],
            reverse=True
        )
        for name, stats in sorted_classes[:5]:
            print(f"    {name}: {stats['percentage']}%")
else:
    print("No test images found in repo. Upload an image to test manually.")
    print(f"Example: curl -X POST {BASE_URL}/analyze -F 'file=@your_image.jpg'")